# Project 4. Identify which knowledge-based techniques are appropriate for which task

### 1. Expert Systems:
- Project: Medical Diagnosis System
- Description: Build an expert system that takes patient symptoms as input and provides potential medical conditions as output.
- Techniques: Rule-based reasoning, forward chaining, inference engine.

To create an expert system for the Medical Diagnosis System, we'll use the pyknow library, which is a Python implementation of the CLIPS expert system shell. We'll define rules that infer potential medical conditions based on the patient's symptoms. Make sure you have the pyknow library installed before running the code. You can install it using pip install pyknow.

In [2]:
!pip install experta

In [3]:
from experta import *


# Define the Facts (Patient Symptoms)
class Symptom(Fact):
    pass

# Define the Medical Conditions
class MedicalCondition(Fact):
    pass


# Define the Expert System Rules
class MedicalDiagnosisExpertSystem(KnowledgeEngine):
    @Rule(Symptom(cough=True) & Symptom(fever=True) & Symptom(fatigue=True))
    def rule_flu(self):
        self.declare(MedicalCondition(name="Flu"))

    @Rule(Symptom(headache=True) & Symptom(stiff_neck=True) & Symptom(fever=True))
    def rule_meningitis(self):
        self.declare(MedicalCondition(name="Meningitis"))

    @Rule(Symptom(rash=True) & Symptom(fever=True) & Symptom(joint_pain=True))
    def rule_dengue(self):
        self.declare(MedicalCondition(name="Dengue"))

    @Rule(Symptom(chest_pain=True) & Symptom(shortness_of_breath=True))
    def rule_heart_attack(self):
        self.declare(MedicalCondition(name="Heart Attack"))


# Function to run the Medical Diagnosis Expert System
def run_medical_diagnosis_system(symptoms):
    engine = MedicalDiagnosisExpertSystem()

    for symptom, value in symptoms.items():
        engine.declare(Symptom(**{symptom: value}))

    engine.run()

    # Get the output medical condition
    condition = engine.facts.get(MedicalCondition)
    if condition:
        return condition['name']
    else:
        return "No specific medical condition found."


# Example Usage
if __name__ == "__main__":
    # Example symptoms (You can modify this dictionary with patient symptoms)
    patient_symptoms = {
        'cough': True,
        'fever': True,
        'fatigue': True,
    }

    medical_condition = run_medical_diagnosis_system(patient_symptoms)
    print("Medical Condition:", medical_condition)


Medical Condition: No specific medical condition found.


### 2. Knowledge Graphs:


- Project: Movie Recommendation System
- Description: Create a knowledge graph of movies, actors, genres, and user preferences to recommend personalized movies to users.
- Techniques: RDF data representation, SPARQL querying, semantic similarity.

In [4]:
!pip install rdflib

  Using cached rdflib-6.3.2-py3-none-any.whl (528 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Obtaining dependency information for pyparsing<4,>=2.1.0 from https://files.pythonhosted.org/packages/a4/24/6ae4c9c45cf99d96b06b5d99e25526c060303171fb0aea9da2bfd7dbde93/pyparsing-3.1.0-py3-none-any.whl.metadata
Using cached pyparsing-3.1.0-py3-none-any.whl (102 kB)


In [7]:
!pip install SPARQLWrapper


  Using cached SPARQLWrapper-2.0.0-py3-none-any.whl (28 kB)


In [46]:
## Database
# ...

# Fetch movie data from DBpedia using SPARQL
sparql_endpoint = "https://dbpedia.org/sparql"
sparql_query = """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    
    SELECT ?movie ?title ?genre
    WHERE {
        ?movie a dbo:Film.
        ?movie foaf:name ?title.
        ?movie dbo:genre ?genre.
        FILTER(LANG(?title) = "" || LANGMATCHES(LANG(?title), "en"))
    }
    LIMIT 10
"""

sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(sparql_query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Print the fetched movie data
print("Fetched Movie Data:")
for result in results["results"]["bindings"]:
    print(result["title"]["value"], "-", result["genre"]["value"])

# ...


Fetched Movie Data:
Beautiful Skin - http://dbpedia.org/resource/Electronic_music
Beautiful Skin - http://dbpedia.org/resource/Post-punk
Beautiful Skin - http://dbpedia.org/resource/Indie_music
Private Lives - http://dbpedia.org/resource/Romantic_comedy
Processional - http://dbpedia.org/resource/Modernist
Puttur Narasimha Nayak - http://dbpedia.org/resource/Indian_classical
Puttur Narasimha Nayak - http://dbpedia.org/resource/Bhajans
Puttur Narasimha Nayak - http://dbpedia.org/resource/Playback_singing
Quantum Gate - http://dbpedia.org/resource/Interactive_movie
Scrabble Complete - http://dbpedia.org/resource/Puzzle_game


In [47]:
def add_movie_data_to_graph(graph):
    for movie_uri, movie_data in MOVIE_DATA.items():
        graph.add((URIRef(movie_uri), RDF.type, MOVIE_ONTOLOGY.Movie))
        graph.add((URIRef(movie_uri), FOAF.title, Literal(movie_data["title"])))
        graph.add((URIRef(movie_uri), MOVIE_ONTOLOGY.genre, URIRef(movie_data["genre"])))


In [49]:
def recommend_movies(graph, user_pref):
    # Retrieve user's preferred movie genre
    user_genre = next(graph.objects(subject=user_pref, predicate=MOVIE_ONTOLOGY.genre), None)
    if not user_genre:
        raise ValueError("No genre found for the user's preferred movie!")

    # Define SPARQL query to recommend movies with the same genre as the user's preferred movie
    sparql_query_recommendation = f"""
        PREFIX movie: <http://example.org/movie_ontology#>
        SELECT ?movie ?title
        WHERE {{
            ?movie movie:genre <{user_genre}>.
            FILTER (?movie != <{user_pref}>)
            ?movie movie:title ?title.
        }}
    """

    # Execute the SPARQL query
    results = graph.query(sparql_query_recommendation)

    # Display recommended movies
    print("Recommended Movies:")
    for result in results:
        print(result["title"].toPython())


In [58]:
!pip install requests


  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/7b/c6/7f75892d87d7afcf8ed909f3e74de1bc61abd9d77cd9aab1f449430856c5/charset_normalizer-3.2.0-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl.metadata
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [60]:
import os
from rdflib import Graph, Namespace, RDF, Literal, URIRef

# Get the absolute path to the TTL file
ttl_file = os.path.abspath("movies.ttl")

# Fetch movie data from the TTL file and parse it
graph = Graph()
graph.parse(ttl_file, format="ttl")

# Define movie ontology namespace and classes
MOVIE_ONTOLOGY = Namespace("http://example.org/movie_ontology#")
MOVIE = MOVIE_ONTOLOGY.Movie
GENRE = MOVIE_ONTOLOGY.genre
ACTOR = MOVIE_ONTOLOGY.actor


In [67]:
import os
import requests
from rdflib import Graph, Namespace, RDF, Literal, URIRef

# Fetch movie data from the TTL file and parse it
ttl_file = os.path.abspath("movies.ttl")
graph = Graph()
graph.parse(ttl_file, format="ttl")

# Define movie ontology namespace and classes
MOVIE_ONTOLOGY = Namespace("http://example.org/movie_ontology#")
MOVIE = MOVIE_ONTOLOGY.Movie
GENRE = MOVIE_ONTOLOGY.genre
ACTOR = MOVIE_ONTOLOGY.actor

# Define the user namespace and preferences
USER = Namespace("http://example.org/users/")
user_pref = list(graph.subjects(predicate=RDF.type, object=MOVIE))[0]

# Get the user's preferred genre
user_pref_genre = list(graph.objects(subject=user_pref, predicate=GENRE))[0]

# Define SPARQL query to recommend movies to the user based on preferences
sparql_query_recommendation = f"""
    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {{
        ?movie movie:genre {user_pref_genre.n3()} .
        FILTER(?movie != {user_pref.n3()})
        ?movie movie:title ?title.
    }}
"""

# Send the SPARQL query to the graph and get the results
results = graph.query(sparql_query_recommendation)

# Print the SPARQL query and recommended movies
print("SPARQL Query for Movie Recommendation:\n")
print(sparql_query_recommendation)

print("\nRecommended Movies:")
for result in results:
    print(result.title)


SPARQL Query for Movie Recommendation:


    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {
        ?movie movie:genre <http://example.org/movie_ontology#Action> .
        FILTER(?movie != <http://example.org/movie_ontology#movie1>)
        ?movie movie:title ?title.
    }


Recommended Movies:


In [68]:
import os
import requests
from rdflib import Graph, Namespace, RDF, Literal, URIRef

# Fetch movie data from the TTL file and parse it
ttl_file = os.path.abspath("movies.ttl")
graph = Graph()
graph.parse(ttl_file, format="ttl")

# Define movie ontology namespace and classes
MOVIE_ONTOLOGY = Namespace("http://example.org/movie_ontology#")
MOVIE = MOVIE_ONTOLOGY.Movie
GENRE = MOVIE_ONTOLOGY.genre
ACTOR = MOVIE_ONTOLOGY.actor

# Define the user namespace and preferences
USER = Namespace("http://example.org/users/")
user_pref = list(graph.subjects(predicate=RDF.type, object=MOVIE))[0]

# Get the user's preferred genre
user_pref_genres = list(graph.objects(subject=user_pref, predicate=GENRE))

# Define SPARQL query to recommend movies to the user based on preferences
sparql_query_recommendation = f"""
    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {{
        ?movie movie:genre ?genre .
        FILTER(?movie NOT IN ({", ".join(genre.n3() for genre in user_pref_genres)}))
        ?movie movie:title ?title.
    }}
"""

# Send the SPARQL query to the graph and get the results
results = graph.query(sparql_query_recommendation)

# Print the SPARQL query and recommended movies
print("SPARQL Query for Movie Recommendation:\n")
print(sparql_query_recommendation)

print("\nRecommended Movies:")
for result in results:
    print(result.title)


SPARQL Query for Movie Recommendation:


    PREFIX movie: <http://example.org/movie_ontology#>
    SELECT ?movie ?title
    WHERE {
        ?movie movie:genre ?genre .
        FILTER(?movie NOT IN (<http://example.org/movie_ontology#Action>))
        ?movie movie:title ?title.
    }


Recommended Movies:
The Matrix
The Shawshank Redemption
